In [ ]:
import pandas as pd
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta, Variable, log, exp, bioDraws, bioMultSum, MonteCarlo, bioNormalCdf, Elem, RandomVariable, Integrate, PanelLikelihoodTrajectory, Derive
import biogeme.database as db
import numpy as np
import biogeme.distributions as dist
import biogeme.exceptions as excep
import biogeme.results as res
import biogeme.loglikelihood as ll
from biogeme.results import bioResults
import biogeme.biogeme_logging as blog
logger = blog.get_screen_logger(level=blog.INFO)
logger.info('info.py')

In [ ]:
df = pd.read_csv('biogeme_data_flat.csv')

In [ ]:
df['lowtreat'] = df['treatment'].replace({0: 1, 1: 0, 2: 0})
df['hightreat'] = df['treatment'].replace({0: 0, 1: 0, 2: 1})
df = df.drop(columns=['person'])

In [ ]:
database = db.Database('df', df)
globals().update(database.variables)

In [ ]:
PR_intercept = Beta('PR_Intercept', 0, None, None, 0)
PR_lowtreat = Beta('PR_lowtreat',0, None, None, 0)
PR_hightreat = Beta('PR_hightreat', 0, None, None, 0)

sigma_s = Beta('sigma_s', 1, None, None, 0)
ec = sigma_s * bioDraws('ec', 'NORMAL_HALTON3')

In [ ]:
PR = (
    PR_intercept
    + PR_hightreat*hightreat
    + PR_lowtreat*lowtreat
   + ec
)

In [ ]:
# measurement model

INTER_Reliab02 = Beta('INTER_Reliab02', 0, None, None, 1)
INTER_Reliab03 = Beta('INTER_Reliab03', 0, None, None, 0)
INTER_Reliab04 = Beta('INTER_Reliab04', 0, None, None, 0)
INTER_Reliab05 = Beta('INTER_Reliab05', 0, None, None, 0)

B_Reliab02_F1 = Beta('B_Reliab02_F1', 1, None, None, 1)
B_Reliab03_F1 = Beta('B_Reliab03_F1', 1, None, None, 0)
B_Reliab04_F1 = Beta('B_Reliab04_F1',1, None, None, 0)
B_Reliab05_F1 = Beta('B_Reliab05_F1', 1, None, None, 0)

MODEL_Reliab02 = INTER_Reliab02 + B_Reliab02_F1 * PR
MODEL_Reliab03 = INTER_Reliab03 + B_Reliab03_F1 * PR
MODEL_Reliab04 = INTER_Reliab04 + B_Reliab04_F1 * PR
MODEL_Reliab05 = INTER_Reliab05 + B_Reliab05_F1 * PR

SIGMA_STAR_Reliab01 = Beta('SIGMA_STAR_Reliab01', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab02 = Beta('SIGMA_STAR_Reliab02', 1, 1.0e-5, None, 1)
SIGMA_STAR_Reliab03 = Beta('SIGMA_STAR_Reliab03', 1, 1.0e-5, None, 0)
SIGMA_STAR_Reliab04 = Beta('SIGMA_STAR_Reliab04', 1, 1.0e-5, None, 0)
SIGMA_STAR_Reliab05 = Beta('SIGMA_STAR_Reliab05', 1, 1.0e-5, None, 0)

delta_1 = Beta('delta_1',1,0,10,0)
delta_2 = Beta('delta_2',1,0,10,0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

# define the indicator thresholds
Reliab02_tau_1 = (tau_1-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_2 = (tau_2-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_3 = (tau_3-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_4 = (tau_4-MODEL_Reliab02) / SIGMA_STAR_Reliab02
Reliab02_tau_5 = (tau_5-MODEL_Reliab02) / SIGMA_STAR_Reliab02

Reliab03_tau_1 = (tau_1-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_2 = (tau_2-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_3 = (tau_3-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_4 = (tau_4-MODEL_Reliab03) / SIGMA_STAR_Reliab03
Reliab03_tau_5 = (tau_5-MODEL_Reliab03) / SIGMA_STAR_Reliab03

Reliab04_tau_1 = (tau_1-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_2 = (tau_2-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_3 = (tau_3-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_4 = (tau_4-MODEL_Reliab04) / SIGMA_STAR_Reliab04
Reliab04_tau_5 = (tau_5-MODEL_Reliab04) / SIGMA_STAR_Reliab04

Reliab05_tau_1 = (tau_1-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_2 = (tau_2-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_3 = (tau_3-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_4 = (tau_4-MODEL_Reliab05) / SIGMA_STAR_Reliab05
Reliab05_tau_5 = (tau_5-MODEL_Reliab05) / SIGMA_STAR_Reliab05

# probit models for indicators
IndReliab02 = {
    1: bioNormalCdf(Reliab02_tau_1),
    2: bioNormalCdf(Reliab02_tau_2)-bioNormalCdf(Reliab02_tau_1),
    3: bioNormalCdf(Reliab02_tau_3)-bioNormalCdf(Reliab02_tau_2),
    4: bioNormalCdf(Reliab02_tau_4)-bioNormalCdf(Reliab02_tau_3),
    5: bioNormalCdf(Reliab02_tau_5)-bioNormalCdf(Reliab02_tau_4),
    6: 1-bioNormalCdf(Reliab02_tau_5),
    9999: 1.0
}

P_Reliab02 = Elem(IndReliab02, reliab_2)

IndReliab03 = {
    1: bioNormalCdf(Reliab03_tau_1),
    2: bioNormalCdf(Reliab03_tau_2)-bioNormalCdf(Reliab03_tau_1),
    3: bioNormalCdf(Reliab03_tau_3)-bioNormalCdf(Reliab03_tau_2),
    4: bioNormalCdf(Reliab03_tau_4)-bioNormalCdf(Reliab03_tau_3),
    5: bioNormalCdf(Reliab03_tau_5)-bioNormalCdf(Reliab03_tau_4),
    6: 1-bioNormalCdf(Reliab03_tau_5),
    9999: 1.0
}

P_Reliab03 = Elem(IndReliab03, reliab_3)

IndReliab04 = {
    1: bioNormalCdf(Reliab04_tau_1),
    2: bioNormalCdf(Reliab04_tau_2)-bioNormalCdf(Reliab04_tau_1),
    3: bioNormalCdf(Reliab04_tau_3)-bioNormalCdf(Reliab04_tau_2),
    4: bioNormalCdf(Reliab04_tau_4)-bioNormalCdf(Reliab04_tau_3),
    5: bioNormalCdf(Reliab04_tau_5)-bioNormalCdf(Reliab04_tau_4),
    6: 1-bioNormalCdf(Reliab04_tau_5),
    9999: 1.0
}

P_Reliab04 = Elem(IndReliab04, reliab_4)

IndReliab05 = {
    1: bioNormalCdf(Reliab05_tau_1),
    2: bioNormalCdf(Reliab05_tau_2)-bioNormalCdf(Reliab05_tau_1),
    3: bioNormalCdf(Reliab05_tau_3)-bioNormalCdf(Reliab05_tau_2),
    4: bioNormalCdf(Reliab05_tau_4)-bioNormalCdf(Reliab05_tau_3),
    5: bioNormalCdf(Reliab05_tau_5)-bioNormalCdf(Reliab05_tau_4),
    6: 1-bioNormalCdf(Reliab05_tau_5),
    9999: 1.0
}

P_Reliab05 = Elem(IndReliab05, reliab_5)

In [ ]:
ASC_EV = Beta('ASC_EV', 0, None, None, 0)
ASC_EV_S = Beta('ASC_EV_S', 1, None, None, 0)
ASC_EV_RND = ASC_EV + ASC_EV_S * bioDraws('ASC_EV_S_RND', 'NORMAL_HALTON3')

B_PRICE_EV = Beta('B_PRICE_EV', 0, None, None, 0)
B_PRICE_CONV = Beta('B_PRICE_CONV', 0, None, None, 0)

B_OPCOST_EV = Beta('B_OPCOST_EV', 0, None, None, 0)
B_OPCOST_CONV = Beta('B_OPCOST_CONV', 0, None, None, 0)

B_RANGE_EV = Beta('B_RANGE_EV', 0, None, None, 0)
B_RANGE_CONV = Beta('B_RANGE_CONV', 0, None, None, 0)

B_LOGCHARGAVAIL_EV = Beta('B_LOGCHARGAVAIL_EV',0, None, None, 0)
B_CHARGAVAIL_EV = Beta('B_CHARGAVAIL_EV',0, None, None, 0)

B_PR_NOHOME = Beta('B_PR_NOHOME', 0, None, None, 0)
B_PR_HOME = Beta('B_PR_HOME', 0, None, None, 0)

In [ ]:
# define the utility functions
V_EV_1 = (
    ASC_EV
    + B_PRICE_EV * (evprice_1/budget)
    + B_OPCOST_EV * evopcost_1
    + B_RANGE_EV * (evrange_1/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_1/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_1 = (
    B_PRICE_CONV * (icevprice_1/budget)
    + B_OPCOST_CONV * icevopcost_1
    + B_RANGE_CONV * (icevrange_1/100)
)

###########################################################################

V_EV_2 = (
    ASC_EV
    + B_PRICE_EV * (evprice_2/budget)
    + B_OPCOST_EV * evopcost_2
    + B_RANGE_EV * (evrange_2/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_2/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_2 = (
    B_PRICE_CONV * (icevprice_2/budget)
    + B_OPCOST_CONV * icevopcost_2
    + B_RANGE_CONV * (icevrange_2/100)
)

###########################################################################
V_EV_3 = (
    ASC_EV
    + B_PRICE_EV * (evprice_3/budget)
    + B_OPCOST_EV * evopcost_3
    + B_RANGE_EV * (evrange_3/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_3/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_3 = (
    B_PRICE_CONV * (icevprice_3/budget)
    + B_OPCOST_CONV * icevopcost_3
    + B_RANGE_CONV * (icevrange_3/100)
)

###########################################################################

V_EV_4 = (
    ASC_EV
    + B_PRICE_EV * (evprice_4/budget)
    + B_OPCOST_EV * evopcost_4
    + B_RANGE_EV * (evrange_4/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_4/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_4 = (
    B_PRICE_CONV * (icevprice_4/budget)
    + B_OPCOST_CONV * icevopcost_4
    + B_RANGE_CONV * (icevrange_4/100)
)

###########################################################################

V_EV_5 = (
    ASC_EV
    + B_PRICE_EV * (evprice_5/budget)
    + B_OPCOST_EV * evopcost_5
    + B_RANGE_EV * (evrange_5/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_5/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_5 = (
    B_PRICE_CONV * (icevprice_5/budget)
    + B_OPCOST_CONV * icevopcost_5
    + B_RANGE_CONV * (icevrange_5/100)
)

###########################################################################

V_EV_6 = (
    ASC_EV
    + B_PRICE_EV * (evprice_6/budget)
    + B_OPCOST_EV * evopcost_6
    + B_RANGE_EV * (evrange_6/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_6/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_6 = (
    B_PRICE_CONV * (icevprice_6/budget)
    + B_OPCOST_CONV * icevopcost_6
    + B_RANGE_CONV * (icevrange_6/100)
)

###########################################################################

V_EV_7 = (
    ASC_EV
    + B_PRICE_EV * (evprice_7/budget)
    + B_OPCOST_EV * evopcost_7
    + B_RANGE_EV * (evrange_7/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_7/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_7 = (
    B_PRICE_CONV * (icevprice_7/budget)
    + B_OPCOST_CONV * icevopcost_7
    + B_RANGE_CONV * (icevrange_7/100)
)

###########################################################################

V_EV_8 = (
    ASC_EV
    + B_PRICE_EV * (evprice_8/budget)
    + B_OPCOST_EV * evopcost_8
    + B_RANGE_EV * (evrange_8/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_8/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_8 = (
    B_PRICE_CONV * (icevprice_8/budget)
    + B_OPCOST_CONV * icevopcost_8
    + B_RANGE_CONV * (icevrange_8/100)
)

###########################################################################

V_EV_9 = (
    ASC_EV
    + B_PRICE_EV * (evprice_9/budget)
    + B_OPCOST_EV * evopcost_9
    + B_RANGE_EV * (evrange_9/100)
   + B_LOGCHARGAVAIL_EV * log(evchargavail_9/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_9 = (
    B_PRICE_CONV * (icevprice_9/budget)
    + B_OPCOST_CONV * icevopcost_9
    + B_RANGE_CONV * (icevrange_9/100)
)

###########################################################################

V_EV_10 = (
    ASC_EV
    + B_PRICE_EV * (evprice_10/budget)
    + B_OPCOST_EV * evopcost_10
    + B_RANGE_EV * (evrange_10/100)
    + B_LOGCHARGAVAIL_EV * log(evchargavail_10/100)
    + B_PR_HOME * PR * homecharge
    + B_PR_NOHOME * PR * (1-homecharge)
)

V_CONV_10 = (
    B_PRICE_CONV * (icevprice_10/budget)
    + B_OPCOST_CONV * icevopcost_10
    + B_RANGE_CONV * (icevrange_10/100)
)

###########################################################################



In [ ]:
V1 = {1: V_EV_1, 2: V_CONV_1}
V2 = {1: V_EV_2, 2: V_CONV_1}
V3 = {1: V_EV_3, 2: V_CONV_3}
V4 = {1: V_EV_4, 2: V_CONV_4}
V5 = {1: V_EV_5, 2: V_CONV_5}
V6 = {1: V_EV_6, 2: V_CONV_6}
V7 = {1: V_EV_7, 2: V_CONV_7}
V8 = {1: V_EV_8, 2: V_CONV_8}
V9 = {1: V_EV_9, 2: V_CONV_9}
V10 = {1: V_EV_10, 2: V_CONV_10}

av1 = {1: ev_av, 2: icev_av}
av2 = {1: ev_av, 2: icev_av}
av3 = {1: ev_av, 2: icev_av}
av4 = {1: ev_av, 2: icev_av}
av5 = {1: ev_av, 2: icev_av}
av6 = {1: ev_av, 2: icev_av}
av7 = {1: ev_av, 2: icev_av}
av8 = {1: ev_av, 2: icev_av}
av9 = {1: ev_av, 2: icev_av}
av10 = {1: ev_av, 2: icev_av}

In [ ]:
prob1 = models.logit(V1, av1, choice1)
prob2 = models.logit(V2, av2, choice2)
prob3 = models.logit(V3, av3, choice3)
prob4 = models.logit(V4, av4, choice4)
prob5 = models.logit(V5, av5, choice5)
prob6 = models.logit(V6, av6, choice6)
prob7 = models.logit(V7, av7, choice7)
prob8 = models.logit(V8, av8, choice8)
prob9 = models.logit(V9, av9, choice9)
prob10 = models.logit(V10, av10, choice10)

probtot = prob1 * prob2 * prob3 * prob4 * prob5 * prob6 * prob7 * prob8 * prob9 * prob10
# obsprob = [prob1, prob2, prob3, prob4, prob5, prob6, prob7, prob8, prob9, prob10]
# # condprobIndiv = exp(bioMultSum(obsprob))
# condprobtot = condprobIndiv * P_Reliab02 * P_Reliab03 * P_Reliab04 * P_Reliab05

condprobtot = probtot * P_Reliab02 * P_Reliab03 * P_Reliab04 * P_Reliab05
logprob = log(MonteCarlo(condprobtot))

In [ ]:
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'BinaryLogit_Flat_11.5_NoLog'

In [ ]:
# run model and print results
model_results = the_biogeme.estimate()
pandasResults = model_results.getEstimatedParameters()
print(pandasResults)